works with https://tech.yandex.ru/maps/geosearch/doc/concepts/request-docpage/

In [1]:
import pandas as pd
import requests
import os

### importing data

In [2]:
# reading api keys
api_keys = pd.read_excel('../api_keys.xlsx')
api_keys.set_index('key_name', inplace=True)

# API Yandex organization search
search_api_key = api_keys.loc['yandex_search']['key']

In [3]:
# coords table
coords = pd.read_excel('./input_params.xlsx', sheet_name = 'coords')
coords.set_index('point_name', inplace=True)

def extract_lon_lat(location):
    loc = coords.loc[location]
    return loc['lon'], loc['lat']

# south-west bounding box coordinates
sw_lon, sw_lat = extract_lon_lat('southwest')

# north-east bounding box coordinates
ne_lon, ne_lat = extract_lon_lat('northeast')

In [4]:
# table of items to request
request_list = pd.read_excel('./input_params.xlsx', sheet_name = 'request_example')


In [5]:
# lists of items without missing values
for item_type in list(request_list.columns):
    list_of_items = request_list[item_type].dropna().to_list()
    print(list_of_items)

['общеобразовательная школа', 'гимназия', 'лицей', 'частная школа', 'школа-интернат', 'детский сад', 'вуз', 'колледж', 'училище', 'художественная школа', 'музыкальная школа']
['поликлиника', 'больница', 'пункт скорой помощи', 'фельдшерско-амбулаторный пункт', 'лаборатория анализов', 'аптека', 'женская консультация', 'стоматологическая клиника', 'травмпункт']
['стадион', 'бассейн', 'каток', 'спортивная школа', 'спортивный клуб', 'спортивная площадка', 'спортивная секция', 'workout', 'футбольное поле', 'теннисный корт', 'хоккейная коробка']
['кинотеатр', 'концертный зал', 'музей', 'выставка', 'галерея', 'дом культуры', 'библиотека', 'театр', 'цирк', 'зоопарк']
['супермаркет', 'магазин продуктов', 'гипермаркет', 'магазин одежды', 'магазин обуви', 'промтовары', 'хозтовары', 'пекарня', 'магазин спортивных товаров', 'магазин электроники', 'салон связи', 'товары для дома', 'алкогольный магазин', 'ювелирный магазин', 'рынок', 'рыбный магазин', 'цветы', 'магазин табака', 'книжный магазин', 'маг

### request

In [7]:
# in case we go out of limit with skip counts

print(sw_lon, sw_lat, ne_lon, ne_lat)

def split_into_quadrants(sw_lon, sw_lat, ne_lon, ne_lat):
    for slon in [sw_lon, (sw_lon + ne_lon) / 2]:
        for slat in [sw_lat, (sw_lat + ne_lat) / 2]:
            yield slon, slat, slon + (ne_lon - sw_lon) / 2, slat + (ne_lat - sw_lat) / 2

print("\n".join(map(str, list(split_into_quadrants(sw_lon, sw_lat, ne_lon, ne_lat)))))

103.810924 52.47304499999999 103.92790600000001 52.57374599999999
(103.810924, 52.47304499999999, 103.869415, 52.52339549999999)
(103.810924, 52.52339549999999, 103.869415, 52.57374599999999)
(103.869415, 52.47304499999999, 103.92790600000001, 52.52339549999999)
(103.869415, 52.52339549999999, 103.92790600000001, 52.57374599999999)


In [8]:
def split_single_query_request(search_query, sw_lon, sw_lat, ne_lon, ne_lat):
    print(f"Splitting {search_query} by 4 at", sw_lon, sw_lat, ne_lon, ne_lat)
    results = []
    for (lon1, lat1, lon2, lat2) in split_into_quadrants(sw_lon, sw_lat, ne_lon, ne_lat):
        results += single_query_request(search_query, lon1, lat1, lon2, lat2)
    return results


def single_query_request(search_query, sw_lon, sw_lat, ne_lon, ne_lat):
    URL = 'https://search-maps.yandex.ru/v1/'
    bbox = f"{sw_lon:.6f},{sw_lat:.6f}~{ne_lon:.6f},{ne_lat:.6f}"
    results = []
    for skip_cnt in [0, 500, 1000]:
        params = { 
            'text' : search_query, 
            'apikey': search_api_key,
            'lang': 'ru_RU',
            'type':'biz',
            'bbox': bbox,
            'rspn': 1,
            'results': 500,
            'skip': skip_cnt
        }
        response = requests.get(URL, params=params)
        response_json = response.json()
        results += response_json['features']
        feature_count = len(response_json['features'])
        if feature_count < 500:
            break
    else:
        return split_single_query_request(search_query, sw_lon, sw_lat, ne_lon, ne_lat)
    return results

In [9]:
def organization_request(item_type):
    # list of queries from column
    list_of_items = request_list[item_type].dropna().to_list()
    
    # request parameters - URL and bounding box
        
    results = []
    
    # request
    for search_query in list_of_items:
        results += single_query_request(search_query, sw_lon, sw_lat, ne_lon, ne_lat)

    # take only unique values is results

    result_dict = {}
    for result in results:
        result_dict[result['properties']['CompanyMetaData']['id']] = result
    results_unique = list(result_dict.values())

    # transform json, delete exceed columns, create category columns
    for res in results_unique:
        res.update(res['properties'])
        res['lon'], res['lat'] = res['geometry']['coordinates']
        res.update(res['CompanyMetaData'])
        if 'class' not in res['Categories'][0]:
            print("No 'class' in category: ", res['Categories'][0])
        res['category'] = res['Categories'][0].get('class', 'UNKNOWN')
        res['category_name'] = res['Categories'][0].get('name', 'UNKNOWN')
        for cat in res['Categories']:
            res['cat_' + cat.get('class', 'UNKNOWN').replace(' ', '_')] = True
        for key in ['properties', 'CompanyMetaData', 'boundedBy', 'type', 'geometry', 'url', 'Phones', 'Hours', 'Categories']:
            if key in res:
                del res[key]

    # create dataframe out of transformed results
    df = pd.DataFrame(results_unique)
    
    return df

### saving result

In [10]:
# create dir for output files
os.makedirs("output", exist_ok=True)

In [66]:
# create csv files for every category in request

for item_type in list(request_list.columns):
    df = organization_request(item_type)
    df.to_csv("./output/{}.csv".format(str(item_type))) 

In [11]:
request_list.columns

Index(['education', 'healthcare', 'sport', 'culture', 'shops', 'services',
       'food', 'leisure', 'parks', 'government', 'transport'],
      dtype='object')

In [13]:
# example for single category

df = organization_request('shops')
df.to_csv('./output/shops.csv')
df.head()

No 'class' in category:  {'name': 'Рыба и морепродукты оптом'}
No 'class' in category:  {'name': 'Рыба и морепродукты оптом'}
No 'class' in category:  {'name': 'Рыба и морепродукты оптом'}


,name,description,lon,lat,id,address,category,category_name,cat_supermarket,cat_malls,...,cat_industrial_enterprise,cat_office_service,cat_fallback_services,cat_clothes_shop,cat_drugstores,cat_mobile_phones,cat_printing_services,cat_furniture_store,cat_haulier,cat_bars
0,СМС,"8, 96-й квартал, Ангарск, Россия",103.878009,52.523231,191284502208,"Россия, Иркутская область, Ангарск, 96-й кварт...",supermarket,Супермаркет,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,На Крупской,"8, 96-й квартал, Ангарск, Россия",103.877926,52.523152,50481882140,"Россия, Иркутская область, Ангарск, 96-й кварт...",supermarket,Супермаркет,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Слата,"33, 13-й микрорайон, Ангарск, Россия",103.864823,52.519649,140423526726,"Россия, Иркутская область, Ангарск, 13-й микро...",supermarket,Супермаркет,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Синенький,"Московская ул., 54, Ангарск, Россия",103.884949,52.538727,1012925912,"Россия, Иркутская область, Ангарск, Московская...",supermarket,Супермаркет,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Дискаунтер Хороший,"26, 13-й микрорайон, Ангарск, Россия",103.867497,52.517773,14617009521,"Россия, Иркутская область, Ангарск, 13-й микро...",supermarket,Супермаркет,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
